In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [62]:
df = pd.read_csv('Dataset/train.csv')

In [66]:
columns = ['damage_grade']
y = df.damage_grade
y = pd.get_dummies(y, dummy_na=True)
print(y[:10])
columns = ['has_geotechnical_risk_fault_crack',
          'has_geotechnical_risk_flood',
          'has_geotechnical_risk_land_settlement',
          'has_geotechnical_risk_landslide',
          'has_geotechnical_risk_liquefaction',
          'has_geotechnical_risk_other',
          'has_geotechnical_risk_rock_fall'
          ]
x = pd.DataFrame(df, columns=columns)

column_has_repair_started = []
for i in df.has_repair_started:
    if i == 0.0 or i == 1.0:
        column_has_repair_started.append(int(i))
    else:
        column_has_repair_started.append(0)
print(column_has_repair_started[:10])
has_repair_started = pd.DataFrame(column_has_repair_started)
x = pd.concat([x, has_repair_started],axis=1) 
x = x.rename(columns={0: 'has_repair_started'})

column_has_geotechnical_risk = []
for i in df.has_geotechnical_risk:
    if i == 0.0 or i == 1.0:
        column_has_geotechnical_risk.append(int(i))
    else:
        column_has_geotechnical_risk.append(0)
print(column_has_geotechnical_risk[:10])
has_geotechnical_risk = pd.DataFrame(column_has_geotechnical_risk)
x = pd.concat([x, has_geotechnical_risk],axis=1) 
x = x.rename(columns={0: 'has_geotechnical_risk'})

has_geotechnical_risk = df.has_geotechnical_risk
has_geotechnical_risk = pd.get_dummies(has_geotechnical_risk, dummy_na=True)
print(has_geotechnical_risk[:10])
x = pd.concat([has_geotechnical_risk, x],axis=1)

district_id = df.district_id
district_id = pd.get_dummies(district_id, dummy_na=True)
print(district_id[:10])
x = pd.concat([district_id, x],axis=1) 

   Grade 1  Grade 2  Grade 3  Grade 4  Grade 5  NaN
0        0        0        0        1        0    0
1        0        1        0        0        0    0
2        1        0        0        0        0    0
3        0        0        0        0        1    0
4        0        0        1        0        0    0
5        0        0        0        0        1    0
6        0        1        0        0        0    0
7        0        0        0        0        1    0
8        1        0        0        0        0    0
9        0        0        1        0        0    0
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
    0.0   1.0  NaN 
0     1     0     0
1     1     0     0
2     1     0     0
3     1     0     0
4     1     0     0
5     1     0     0
6     0     1     0
7     1     0     0
8     1     0     0
9     1     0     0
    7.0    9.0    10.0   11.0   12.0   13.0   20.0   21.0   22.0   23.0  \
0      0      0      0      0      0      0      0      0      0      0

In [67]:
print(df.dtypes)

area_assesed                              object
building_id                               object
damage_grade                              object
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object


In [68]:
print(x.shape)

(631761, 44)


In [69]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [79]:
model = Sequential()
model.add(Dense(44, input_dim=44, activation='relu'))
model.add(Dense(29, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(17, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 44)                1980      
_________________________________________________________________
dense_35 (Dense)             (None, 29)                1305      
_________________________________________________________________
dropout_12 (Dropout)         (None, 29)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 17)                510       
_________________________________________________________________
dropout_13 (Dropout)         (None, 17)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 6)                 108       
Total params: 3,903
Trainable params: 3,903
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
train_data = x[:500000]
train_labels = y[:500000]
test_data = x[500000:]
test_labels = y[500000:]

In [81]:
model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=10, batch_size=100)
scores = model.evaluate(test_data, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
model.save('my_model.h5')

Train on 500000 samples, validate on 131761 samples
Epoch 1/10
500000/500000 [==============================] - 24s 47us/step - loss: 1.3501 - acc: 0.4185 - val_loss: 1.3248 - val_acc: 0.4320
Epoch 2/10
500000/500000 [==============================] - 22s 44us/step - loss: 1.3328 - acc: 0.4265 - val_loss: 1.3245 - val_acc: 0.4321
Epoch 3/10
500000/500000 [==============================] - 21s 41us/step - loss: 1.3306 - acc: 0.4281 - val_loss: 1.3226 - val_acc: 0.4324
Epoch 4/10
500000/500000 [==============================] - 21s 42us/step - loss: 1.3295 - acc: 0.4282 - val_loss: 1.3204 - val_acc: 0.4314
Epoch 5/10
500000/500000 [==============================] - 22s 44us/step - loss: 1.3285 - acc: 0.4283 - val_loss: 1.3200 - val_acc: 0.4312
Epoch 6/10
500000/500000 [==============================] - 21s 42us/step - loss: 1.3284 - acc: 0.4286 - val_loss: 1.3206 - val_acc: 0.4320
Epoch 7/10
500000/500000 [==============================] - 21s 43us/step - loss: 1.3280 - acc: 0.4293 - val

In [46]:
df = pd.read_csv('Dataset/test.csv')
columns = ['has_geotechnical_risk_fault_crack',
          'has_geotechnical_risk_flood',
          'has_geotechnical_risk_land_settlement',
          'has_geotechnical_risk_landslide',
          'has_geotechnical_risk_liquefaction',
          'has_geotechnical_risk_other',
          'has_geotechnical_risk_rock_fall'
          ]
xnew = pd.DataFrame(df, columns=columns)

column_has_repair_started = []
for i in df.has_repair_started:
    if i == 0.0 or i == 1.0:
        column_has_repair_started.append(int(i))
    else:
        column_has_repair_started.append(0)
has_repair_started = pd.DataFrame(column_has_repair_started)
xnew = pd.concat([xnew, has_repair_started],axis=1) 
xnew = xnew.rename(columns={0: 'has_repair_started'})

column_has_geotechnical_risk = []
for i in df.has_geotechnical_risk:
    if i == 0.0 or i == 1.0:
        column_has_geotechnical_risk.append(int(i))
    else:
        column_has_geotechnical_risk.append(0)
has_geotechnical_risk = pd.DataFrame(column_has_geotechnical_risk)
xnew = pd.concat([has_geotechnical_risk, xnew],axis=1) 
xnew = xnew.rename(columns={0: 'has_geotechnical_risk'})

has_geotechnical_risk = df.has_geotechnical_risk
has_geotechnical_risk = pd.get_dummies(has_geotechnical_risk, dummy_na=True)
xnew = pd.concat([has_geotechnical_risk, xnew],axis=1)

district_id = df.district_id
district_id = pd.get_dummies(district_id, dummy_na=True)
xnew = pd.concat([district_id, xnew],axis=1) 

In [47]:
ynew = model.predict(xnew)

In [48]:
print(ynew[:20])

[[  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.49538964e-01   3.12375098e-01   3.64545494e-01   3.76503021e-01
    8.56564522e-01   2.90691832e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01   2.91241467e-01   3.84356439e-01
    9.15508449e-01   3.85149634e-10]
 [  1.35415509e-01   1.66470096e-01  